In [1]:
# Import all the libraries
import pandas as pd
import glob
import os
import re
from pprint import pprint

In [2]:
#Get all CSV files in the folder of GA
all_files = glob.glob(r"C:\Huy Phan\College\VoterTurnout\data\TN\*.csv")

# Files that contain both 'precinct' and 'general' in the filename
general_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'general'])
]


# Files that contain both 'precinct' and 'primary' in the filename
primary_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'primary', 'president'])
]


In [3]:
print("General files:")
for f in general_files:
    print(f)


General files:
C:\Huy Phan\College\VoterTurnout\data\TN\20161108__tn__general__precinct.csv


In [4]:
print("\nPrimary files:")
for f in primary_files:
    print(f)


Primary files:
C:\Huy Phan\College\VoterTurnout\data\TN\20160301__tn__primary__president__precinct.csv


In [5]:
# Process primary files
primary_df_list = []

for file in primary_files:
    try:
        df = pd.read_csv(file)
        
            
        # Combine precinct as county + precinct
        df["precinct"] = df["county"].astype(str) + df["precinct"].astype(str)
        df["precinct"] = df["precinct"].str.upper()
        df["county"] = df["county"].str.upper()

        # Cleaning maybe total precinct
        if 'precinct' in df.columns:
            df['precinct'] = df['precinct'].astype(str)                  
            df = df[df['precinct'].str.upper() != "TOTAL"]               
            df = df[df['precinct'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['precinct'].str.upper() != "NAN"]                
            df = df[~df['precinct'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]   

        if 'county' in df.columns:
            df['county'] = df['county'].astype(str)                  
            df = df[df['county'].str.upper() != "TOTAL"]               
            df = df[df['county'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['county'].str.upper() != "NAN"]       
            df = df[~df['county'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]               
        # Filtering out only President 
        if 'office' in df.columns:
            df = df[df["office"].str.upper() == "PRESIDENTIAL PREFERENCE"]
            
        # Dropping duplicates value
        df = df.drop_duplicates()
        
        primary_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
pri_combined_df = pd.concat(primary_df_list, ignore_index=True)



In [6]:
## Cleaning precinct, county name
# Fixing Cubbler
# pri_combined_df.loc[
#     ( 
#         pri_combined_df["county"].str.contains("\w*F120\w*", case=False, na=False)|
#         pri_combined_df["county"].str.contains("\w*STONE\w*", case=False, na=False)|
#         pri_combined_df["county"].str.contains("\w*OBBLER\w*", case=False, na=False),
#     "county")
# ] = "STONE"

In [7]:
# Checking any suspicious precinct, county
suspicious_precincts = pri_combined_df[
    pri_combined_df["precinct"].isna() |
    pri_combined_df["precinct"].str.strip().str.upper().str.contains(r"\w*TOTAL\w*", na=False) |
    pri_combined_df["precinct"].str.strip().str.upper().isin(["NONE", "NAN"]) |
    pri_combined_df["county"].isna() |
    pri_combined_df["county"].str.strip().str.upper().str.contains(r"\w*TOTAL\w*", na=False) |
    pri_combined_df["county"].str.strip().str.upper().isin(["NONE", "NAN"])
]

In [8]:
# Select only the relevant columns
primary_data = pri_combined_df[["precinct", "party", "candidate", "votes"]]

primary_data

,precinct,party,candidate,votes
0,ANDERSONANDERSONVILLE,Republican,Jeb Bush,2
1,ANDERSONANDERSONVILLE,Republican,Rand Paul,1
2,ANDERSONANDERSONVILLE,Democratic,Hillary Clinton,64
3,ANDERSONBRICEVILLE,Republican,Jeb Bush,0
4,ANDERSONBRICEVILLE,Republican,Rand Paul,0
...,...,...,...,...
38115,WILSON24-1,Republican,George Pataki,1
38116,WILSON25-1,Republican,George Pataki,0
38117,WILSON25-2,Republican,George Pataki,0
38118,WILSONABSENTEE,Republican,George Pataki,0


In [9]:
# Viewing candidate data
primary_data["candidate"].value_counts(dropna=False)

candidate
Uncommitted           4008
Jeb Bush              2008
Rand Paul             2008
John R. Kasich        2008
Mike Huckabee         2008
Lindsey O. Graham     2008
Jim Gilmore           2008
Carly Fiorina         2008
Donald J. Trump       2008
Ted Cruz              2008
Rick Santorum         2008
Chris Christie        2008
Marco Rubio           2008
Ben Carson            2008
George Pataki         2008
Bernie Sanders        2000
Martin J. O'Malley    2000
Hillary Clinton       2000
Name: count, dtype: int64

In [10]:
# Cleaning Candidates

# Turning all primary data to uppercase
primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()

unwanted_keywords = [
    r"\w*VOTE\w*",
    r"\w*UNCOM\w*",
    r"\w*TOTAL\w*",
    r"Blank Votes", r"All Others",r"Total Votes Cast",r"No Preference",
    r"UNCOMMITTED"
]

pattern = "|".join(unwanted_keywords)

# # Assuming candidate column is already string and uppercase
primary_data = primary_data[~primary_data["candidate"].str.upper().str.contains(pattern, regex=True, na=False)]

#fIXING DE LA FUENTE
# primary_data.loc[
#     (
#         primary_data["candidate"].str.contains("rocky.*fuente", case=False, na=False) |
#         primary_data["candidate"].str.contains("LA FUENTE", case=False, na=False) |
#         primary_data["candidate"].str.contains("\w*Fuque\w*", case=False, na=False)
#     ),
#     "candidate"
# ] = "LA FUENTE"


# primary_data.loc[:, "candidate"] = primary_data["candidate"].replace({
#     "Donald I. Trump": "Donald J. Trump",
# })

primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()
# # Fixing the , candidate
# primary_data["candidate"] = (
#     primary_data["candidate"].str.split(",")
#     .str[0]
#     .str.strip()
#     )

# #Selecting only last name
primary_data["candidate"] = primary_data["candidate"].str.split().str[-1]

# Fixing Christie
primary_data.loc[
    primary_data["candidate"].str.contains("CHRISTLE", case=False, na=False),
    "candidate"
] = "CHRISTIE"

# Fixing KASICH
primary_data.loc[
    primary_data["candidate"].str.contains("KAISCH", case=False, na=False),
    "candidate"
] = "CHRISTIE"

# Fixing O'MALLEY
primary_data.loc[
    primary_data["candidate"].str.contains("O'MALLEY", case=False, na=False),
    "candidate"
] = "O'MAILEY"

# Fixing O'MALLEY
primary_data.loc[
    primary_data["candidate"].str.contains("PATAKL", case=False, na=False),
    "candidate"
] = "PATAKI"

# Fixing O'MALLEY
primary_data.loc[
    primary_data["candidate"].str.contains("RUBLO", case=False, na=False),
    "candidate"
] = "RUBIO"

# # Fixing Huckabee
# primary_data.loc[
#     primary_data["candidate"].str.contains("\w*ABEE\w*", case=False, na=False),
#     "candidate"
# ] = "HUCKABEE"

# # Fixing Fiorina
# primary_data.loc[
#     primary_data["candidate"].str.contains("\w*ORINA\W*", case=False, na=False),
#     "candidate"
# ] = "FIORINA"

# # Fixing Gray
# primary_data.loc[
#     primary_data["candidate"].str.contains("GREY", case=False, na=False),
#     "candidate"
# ] = "GRAY"

# # Fixing Wilson
# primary_data.loc[
#     primary_data["candidate"].str.contains("WISON", case=False, na=False),
#     "candidate"
# ] = "WILSON"


# # Fixing separator 
# primary_data["candidate"] = (
#     primary_data["candidate"]
#     .str.split(r"\s*(?:and|/|&|–|-|\+)\s*", n=1, expand=True)[0]
#     .str.strip()
#     .str.upper()
# )

# # Fixing McMullin

# primary_data.loc[
#     (
#         primary_data["candidate"].str.contains("MCMULLIN", case=False, na=False) |
#         primary_data["candidate"].str.contains("EVAN MCMULLEN", case=False, na=False)
#     ),
#     "candidate"
# ] = "MCMULLIN"

# primary_data.loc[
#     primary_data["candidate"].str.contains("De La Fuen", case=False, na=False),
#     "candidate"
# ] = "FUENTE D"

## SPLITTINNG BY ,
# primary_data.loc[:,"candidate"] = (
#     primary_data["candidate"].str.split().str[0].str.upper()
# )

primary_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_33580\4226155669.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_33580\4226155669.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_33580\4226155669.py:43: SettingWithCopyWarning: 
A value is trying to be set on a cop

candidate
BUSH        2008
CRUZ        2008
KASICH      2008
HUCKABEE    2008
GRAHAM      2008
GILMORE     2008
FIORINA     2008
TRUMP       2008
PATAKI      2008
PAUL        2008
SANTORUM    2008
CHRISTIE    2008
RUBIO       2008
CARSON      2008
O'MAILEY    2000
CLINTON     2000
SANDERS     2000
Name: count, dtype: int64

In [11]:
# Viewing Party
primary_data["party"].value_counts(dropna=False)

party
Republican    28112
Democratic     6000
Name: count, dtype: int64

In [12]:
#=====================================
# This function is used to look up party
# for the candidate in the same df
#=====================================
def fill_party_from_data(row, df):
    if pd.notna(row["party"]):
        return row["party"]
    
    # Try to find other rows with the same candidate and known party
    matches = df[(df["candidate"] == row["candidate"]) & (df["party"].notna())]
    if not matches.empty:
        return matches["party"].iloc[0]  # Return the first match's party
    else:
        return None  # Still unknown

#=====================================
# Example usage (uncomment and modify for your workflow)
#=====================================
# Fill party using internal data
# primary_data["party"] = primary_data.apply(
#     lambda row: fill_party_from_data(row, primary_data),
#     axis=1
# )

primary_data["party"].value_counts(dropna=False)


party
Republican    28112
Democratic     6000
Name: count, dtype: int64

In [13]:
# Cleaning Party
# Turning all general data party to uppercase
primary_data["party"] = primary_data["party"].astype(str).str.upper()

primary_data["party"] = primary_data.apply(
    lambda row: fill_party_from_data(row, primary_data),
    axis=1
)
primary_data["party"] = (
    primary_data["party"]
    .replace({
        "REPUBLICAN": "REP",
        "R": "REP",
        "DEMOCRATIC": "DEM",
        "DEMOCRAT":"DEM",
        "DEM": "DEM",
        "D": "DEM",
        "GREEN": "GRN",
        "G": "GREEN",
        "GREEN-RAINBOW":"GRN",
        "LIBERTARIAN": "LIB",
        "LBT": "LIB",
        "L": "LIB",
        "CONSTITUTION": "CON",
        "NP": "IND",
        "NON": "IND",
        "WRI": "IND",
        "WRITE-IN": "IND",
        "ONA": "IND",
        "GEN": "IND",
        "NONE":"UNK"
    })
)
primary_data["party"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_33580\2220816968.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["party"] = primary_data["party"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_33580\2220816968.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["party"] = primary_data.apply(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_33580\2220816968.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

party
REP    28112
DEM     6000
Name: count, dtype: int64

In [14]:
primary_data.loc[:,"candidate_column"] = (
    "pri_" +
    primary_data["party"].str.lower() + "_" +
    primary_data["candidate"]
)

# pivot the table
primary_result = primary_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

primary_result

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_33580\2521670330.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data.loc[:,"candidate_column"] = (


candidate_column,precinct,pri_dem_CLINTON,pri_dem_O'MAILEY,pri_dem_SANDERS,pri_rep_BUSH,pri_rep_CARSON,pri_rep_CHRISTIE,pri_rep_CRUZ,pri_rep_FIORINA,pri_rep_GILMORE,pri_rep_GRAHAM,pri_rep_HUCKABEE,pri_rep_KASICH,pri_rep_PATAKI,pri_rep_PAUL,pri_rep_RUBIO,pri_rep_SANTORUM,pri_rep_TRUMP
0,ANDERSONANDERSONVILLE,64,1,39,2,55,3,170,0,0,0,4,24,0,1,120,0,304
1,ANDERSONBRICEVILLE,11,1,10,0,4,0,44,0,0,0,0,2,0,0,10,0,58
2,ANDERSONBULL RUN,106,5,102,7,79,1,196,1,1,0,1,31,0,3,162,1,316
3,ANDERSONCLAXTON,94,3,58,5,40,2,187,2,0,0,2,23,0,1,118,1,282
4,ANDERSONCLINTON,159,2,94,10,77,0,154,2,0,0,4,44,0,3,201,1,326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006,WILSON24-1,129,1,88,8,42,0,135,1,0,0,3,31,1,1,123,1,203
2007,WILSON25-1,86,0,55,6,28,0,192,0,0,0,1,19,0,1,122,0,219
2008,WILSON25-2,63,1,31,3,20,1,182,0,0,0,0,23,0,2,81,0,192
2009,WILSONABSENTEE,99,4,41,27,10,7,60,2,1,1,0,13,0,2,44,0,108


In [15]:
# Process general files
general_df_list = []

for file in general_files:
    try:
        df = pd.read_csv(file)
        
        # Combine precinct as county + precinct
        df["precinct"] = df["county"].astype(str) + df["precinct"].astype(str)
        df["precinct"] = df["precinct"].str.upper()

        # Cleaning maybe total precinct
        if 'precinct' in df.columns:
            df['precinct'] = df['precinct'].astype(str)                  
            df = df[df['precinct'].str.upper() != "TOTAL"]               
            df = df[df['precinct'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['precinct'].str.upper() != "NAN"]                
        
        if 'county' in df.columns:
            df['county'] = df['county'].astype(str)                  
            df = df[df['county'].str.upper() != "TOTAL"]               
            df = df[df['county'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['county'].str.upper() != "NAN"]       
            df = df[~df['county'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]      
            
        # Filtering out only President 
        if 'office' in df.columns:
            df = df[df["office"].str.upper() == "PRESIDENT"]
        
        # Dropping duplicates value
        df = df.drop_duplicates()
        
        general_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
gen_combined_df = pd.concat(general_df_list, ignore_index=True)

# Checking any suspicious precinct
suspicious_precincts = gen_combined_df[
    gen_combined_df["precinct"].str.strip().str.upper().isin(["TOTAL", "NAN"])
]
general_files


['C:\\Huy Phan\\College\\VoterTurnout\\data\\TN\\20161108__tn__general__precinct.csv']

In [16]:
# Select only the relevant columns
general_data = gen_combined_df[["precinct", "party", "candidate", "votes"]]

general_data

,precinct,party,candidate,votes
0,ANDERSONANDERSONVILLE,Republican,Donald J. Trump,1222.0
1,ANDERSONBRICEVILLE,Republican,Donald J. Trump,243.0
2,ANDERSONBULL RUN,Republican,Donald J. Trump,1397.0
3,ANDERSONCLINTON,Republican,Donald J. Trump,1330.0
4,ANDERSONCLINTON HIGH,Republican,Donald J. Trump,792.0
...,...,...,...,...
16179,WILSON1-1,Independent,Write-In - Tom Hoefling,1.0
16180,WILSON11-1,Independent,Write-In - Tom Hoefling,2.0
16181,WILSON14-1,Independent,Write-In - Tom Hoefling,1.0
16182,WILSON16-1,Independent,Write-In - Tom Hoefling,1.0


In [17]:
# Viewing candidate data
general_data["candidate"].value_counts(dropna=False)

candidate
Donald J. Trump                  2022
Hillary Clinton                  2022
"Rocky" Roque De La Fuente       2022
Gary Johnson                     2022
Alyson Kennedy                   2022
Mike Smith                       2022
Jill Stein                       2022
Write-In - Evan McMullin         1276
Write-In - Darrell L. Castle      623
Write-In - Tom Hoefling            74
Write-In - David Limbaugh          31
Write-In - Laurence Kotlikoff      18
Write-In - Cherunda Fox             6
Write-In - Marshall Schoenke        2
Name: count, dtype: int64

In [18]:
# Cleaning Candidates

# Turning all general data to uppercase
general_data["candidate"] = general_data["candidate"].astype(str).str.upper()


unwanted_keywords = [
    r"\w*VOTE\w*",
    r"\w*UNCOM\w*",
    r"\w*TOTAL\w*",
    r"NAN",
    r"UNCERTIFIED",
    r"UNVERIFIED",
    r"NONE OF THE ABOVE",
    r"LBT",
    r"DEM",
    r"REP",
    r"GRN",
    r"NAN"

]

# Fixing FUENTE
general_data.loc[
    general_data["candidate"].str.contains("\w*Fuenta\w*", case=False, na=False),
    "candidate"
] = "FUENTE"
pattern = "|".join(unwanted_keywords)

# # FILTER OUT TRASH WORDS
general_data = general_data[~general_data["candidate"].str.upper().str.contains(pattern, regex=True, na=False)]

# # Fixing Darrell Castle"
# general_data.loc[
#     general_data["candidate"].str.contains("\w*ASTLE\w*", case=False, na=False),
#     "candidate"
# ] = "CASTLE"

# general_data.loc[
#     general_data["candidate"].str.contains("\w*ATUREN\w*", case=False, na=False),
#     "candidate"
# ] = "MATUREN"

# Fixing mixed president + vice_president by /
general_data["candidate"] = (
    general_data["candidate"].str.split("/")
    .str[0]
    .str.strip()
    )

# #Selecting only last name
# general_data["candidate"] = general_data["candidate"].str.split().str[-1]

# # Fixing the , candidate
# # general_data["candidate"] = (
# #     general_data["candidate"].str.split(",")
# #     .str[0]
# #     .str.strip()
# #     )




# # Fixing Cubbler
# general_data.loc[
#     ( 
#         general_data["candidate"].str.contains("\w*UBBIER\w*", case=False, na=False)|
#         general_data["candidate"].str.contains("\w*UBLER\w*", case=False, na=False)|
#         general_data["candidate"].str.contains("\w*OBBLER\w*", case=False, na=False),
#     "candidate")
# ] = "CUBBLER"


# # Fixing Kotlikoff
# general_data.loc[
#     general_data["candidate"].str.contains("\w*TIKOFF\w*", case=False, na=False),
#     "candidate"
# ] = "KOTLIKOFF"

# # Fixing Valdivia
# general_data.loc[
#     general_data["candidate"].str.contains("\w*VALDIVA\W*", case=False, na=False),
#     "candidate"
# ] = "VALDIVIA"

# # Fixing HOEFLING
# general_data.loc[
#     general_data["candidate"].str.contains("\w*HOEFFLING", case=False, na=False),
#     "candidate"
# ] = "HOEFLING"


# # Fixing McMullin
# general_data.loc[
#     general_data["candidate"].str.contains("MCMULLEN", case=False, na=False) ,
#     "candidate"
# ] = "MCMULLIN"

# Fixing LASTNAME + First name Initial
# general_data.loc[:,"candidate"] = (
#     general_data["candidate"].str.split().str[0].str.upper()
# )


# #Selecting only last name
general_data["candidate"] = general_data["candidate"].str.split().str[-1]

general_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_33580\4103686476.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate"] = general_data["candidate"].astype(str).str.upper()


candidate
TRUMP        2022
CLINTON      2022
FUENTE       2022
JOHNSON      2022
KENNEDY      2022
SMITH        2022
STEIN        2022
MCMULLIN     1276
CASTLE        623
HOEFLING       74
LIMBAUGH       31
KOTLIKOFF      18
FOX             6
SCHOENKE        2
Name: count, dtype: int64

In [19]:
# Viewing Party
general_data["party"].value_counts(dropna=False)

party
Independent    10250
Republican      2022
Democratic      2022
NaN             1890
Name: count, dtype: int64

In [20]:
#=====================================
# This function is used to look up party
# for the candidate in the same df
#=====================================
def fill_party_from_data(row, df):
    if pd.notna(row["party"]):
        return row["party"]
    
    # Try to find other rows with the same candidate and known party
    matches = df[(df["candidate"] == row["candidate"]) & (df["party"].notna())]
    if not matches.empty:
        return matches["party"].iloc[0]  # Return the first match's party
    else:
        return None  # Still unknown
    

#=====================================
# Function to fill remaining missing party from master lookup
#=====================================
def fill_party_from_master(df, master_df):
    party_map = master_df.set_index("candidate")["party"].to_dict()
    df["party"] = df.apply(
        lambda row: party_map.get(row["candidate"], row["party"])
        if pd.isna(row["party"]) else row["party"],
        axis=1
    )
    return df


#=====================================
# Function to update the master candidate-party CSV
#=====================================
def update_master_candidate_party(df, master_path):
    # Read the existing master file
    master_df = pd.read_csv(master_path)

    # Filter and combine only known party entries (exclude 'UNK')
    new_data = (
        df[["candidate", "party"]]
        .dropna()
        .query('party != "UNK"')
        .drop_duplicates()
    )

    # Merge with master and remove duplicates by candidate
    updated_master = pd.concat([master_df, new_data]).drop_duplicates(subset="candidate")

    # Save updated version
    updated_master.to_csv(master_path, index=False)


#=====================================
# Example usage (uncomment and modify for your workflow)
#=====================================
# Fill party using internal data
general_data["party"] = general_data.apply(
    lambda row: fill_party_from_data(row, general_data),
    axis=1
)

# # Fill remaining party using general master CSV
master_party_df = pd.read_csv(r"C:\Huy Phan\College\VoterTurnout\data\cleaned_data\gen_can_party.csv") # USE YOUR OWN ADDRESS
general_data = fill_party_from_master(general_data, master_party_df)
general_data["party"].value_counts(dropna=False)


party
Independent    12107
Republican      2022
Democratic      2022
None              31
IND                2
Name: count, dtype: int64

In [21]:
# Cleaning Party
# Turning all general data party to uppercase
general_data["party"] = general_data["party"].astype(str).str.upper()

general_data["party"] = (
    general_data["party"]
    .replace({
        "REPUBLICAN": "REP",
        "R": "REP",
        "DEMOCRATIC": "DEM",
        "DEMOCRAT":"DEM",
        "DEM": "DEM",
        "D": "DEM",
        "GREEN": "GRN",
        "G": "GRN",
        "GREEN AND RAINBOW":"GRN",
        "GREEN-RAINBOW":"GRN",
        "LIBERTARIAN": "LIB",
        "LIBERTARIN":"LIB",
        "LBT": "LIB",
        "L": "LIB",
        "CONSTITUTION": "CON",
        "AMERICAN DELTA":"AMD",
        "PROHIBITION":"PRO",
        "NP": "IND",
        "NON": "IND",
        "WRI": "IND",
        "WRITE-IN": "IND",
        "ONA": "IND",
        "GEN": "IND",
        "NONE":"UNK",
        "NPA":"IND",
        "UST":"CON",
        "INDEPENDENT":"IND",
        "(WRITE-IN)":"IND"
    })
)

general_data["party"].value_counts(dropna=False)

party
IND    12109
REP     2022
DEM     2022
UNK       31
Name: count, dtype: int64

In [22]:
# UPDATE MASTER FILE, CAREFUL
update_master_candidate_party(general_data, r"C:\Huy Phan\College\VoterTurnout\data\cleaned_data\gen_can_party.csv")

In [23]:
general_data["candidate_column"] = (
    "gen_" +
    general_data["party"].str.lower() + "_" +
    general_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
general_result = general_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

general_result

candidate_column,precinct,gen_dem_CLINTON,gen_ind_CASTLE,gen_ind_FOX,gen_ind_FUENTE,gen_ind_HOEFLING,gen_ind_JOHNSON,gen_ind_KENNEDY,gen_ind_KOTLIKOFF,gen_ind_MCMULLIN,gen_ind_SCHOENKE,gen_ind_SMITH,gen_ind_STEIN,gen_rep_TRUMP,gen_unk_LIMBAUGH
0,ANDERSONANDERSONVILLE,227.0,1.0,0.0,2.0,0.0,32.0,1.0,0.0,3.0,0.0,7.0,5.0,1222.0,0.0
1,ANDERSONBRICEVILLE,42.0,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,1.0,0.0,243.0,0.0
2,ANDERSONBULL RUN,424.0,7.0,0.0,3.0,0.0,76.0,2.0,0.0,2.0,0.0,13.0,6.0,1397.0,0.0
3,ANDERSONCLAXTON,387.0,3.0,0.0,2.0,0.0,53.0,0.0,0.0,5.0,0.0,8.0,5.0,1319.0,0.0
4,ANDERSONCLINTON,556.0,0.0,0.0,5.0,0.0,47.0,2.0,0.0,4.0,0.0,19.0,8.0,1330.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,WILSON8-1,691.0,0.0,0.0,3.0,0.0,77.0,4.0,0.0,5.0,0.0,12.0,17.0,1512.0,0.0
2018,WILSON9-1,400.0,0.0,0.0,2.0,0.0,36.0,1.0,0.0,1.0,0.0,9.0,22.0,852.0,0.0
2019,WILSON9-2,103.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0,9.0,370.0,0.0
2020,WILSONABSENTEE,397.0,0.0,0.0,4.0,0.0,47.0,3.0,0.0,14.0,0.0,9.0,7.0,661.0,0.0


In [24]:
# Merge
combined = pd.merge(primary_result, general_result, on="precinct", how="inner")

# Convert DEM primary columns to numeric and calculate total
dem_cols = combined.filter(like="pri_dem_").columns
combined[dem_cols] = combined[dem_cols].apply(pd.to_numeric, errors="coerce")
combined["dem_primary_total"] = combined[dem_cols].sum(axis=1)

# Convert general election columns to numeric and calculate total
gen_cols = combined.filter(like="gen_").columns
combined[gen_cols] = combined[gen_cols].apply(pd.to_numeric, errors="coerce")
combined["general_total"] = combined[gen_cols].sum(axis=1)

# Convert all numeric columns to int
for col in combined.columns[1:]:
    combined[col] = pd.to_numeric(combined[col], errors='coerce').fillna(0).astype(int)

# Identify filtered-out precincts
primary_precincts = set(primary_result["precinct"])
general_precincts = set(general_result["precinct"])
combined_precincts = set(combined["precinct"])

primary_filtered_out = primary_precincts - combined_precincts
general_filtered_out = general_precincts - combined_precincts




In [25]:


combined.to_csv("TN.csv", index=False)
# Step 3: Identify unmatched precincts and retrieve full rows
combined_precincts = set(combined["precinct"])

# Rows in primary_result but not in combined
primary_filtered_out = primary_result[~primary_result["precinct"].isin(combined_precincts)]
primary_filtered_out.to_csv("TN_primary_filtered.csv", index=False)

# Rows in general_result but not in combined
general_filtered_out = general_result[~general_result["precinct"].isin(combined_precincts)]
general_filtered_out.to_csv("TN_general_filtered.csv", index=False)

In [26]:
print(f"primary: {len(primary_precincts)}, general: {len(general_precincts)}, combined: {len(combined)}")

primary: 2011, general: 2022, combined: 1703


In [27]:
print(f"primary: {len(pri_combined_df['county'].unique())}, general: {len(gen_combined_df['county'].unique())}")


primary: 95, general: 95


In [28]:
pri_combined_df['precinct'].unique()

array(['ANDERSONANDERSONVILLE', 'ANDERSONBRICEVILLE', 'ANDERSONBULL RUN',
       ..., 'WILSON25-2', 'WILSONABSENTEE', 'WILSONPROVISIONAL'],
      dtype=object)

In [29]:
gen_combined_df['precinct'].unique()

array(['ANDERSONANDERSONVILLE', 'ANDERSONBRICEVILLE', 'ANDERSONBULL RUN',
       ..., 'WILSON25-2', 'WILSONPROVISIONAL', 'WILSONABSENTEE'],
      dtype=object)

In [30]:
primary_counties = set(pri_combined_df['precinct'].dropna().str.strip().str.upper())
general_counties = set(gen_combined_df['precinct'].dropna().str.strip().str.upper())

diff = primary_counties - general_counties
print(f"Counties in primary but not in general: {len(diff)}")
print(diff)


Counties in primary but not in general: 308
{'KNOX69S BLUEGRASS', 'HAYWOOD1-MAY', 'DAVIDSON4-MAR', 'WILLIAMSON3-AUG', 'MONROEVONORE CC', 'DAVIDSON3-OCT', 'DAVIDSON3-DEC', 'RUTHERFORD1-AUG', 'RUTHERFORD1-MAR', 'JEFFERSON3 WHITE PINE-2', 'JACKSON1-JAN', 'DAVIDSON2-AUG', 'DAVIDSON2-JUN', 'MONROEBROAD ST', 'MONROEBALLPLAY', 'COFFEE17 HANDS ON SCIENCE', 'LEWIS1-JUN', 'MACONCOUNTY WELCOME CENTER', 'GRAINGER2-JAN', 'DICKSON2-FEB', 'WILLIAMSON2-SEP', 'WILLIAMSON5-SEP', 'CROCKETT4', 'HAYWOOD1-OCT', 'DAVIDSON3-FEB', 'DAVIDSON1-MAR', 'WILSON1-NOV', 'HAMBLEN1-MAY', 'MAURYWEST END 1-2-7', 'HAYWOOD1-JAN', 'HAMBLEN1-SEP', 'DAVIDSON3-APR', 'JOHNSON7 HIGH SCHOOL', 'DICKSON2-JAN', 'RUTHERFORD2-APR', 'WILLIAMSON2-NOV', 'HAMBLEN1-OCT', 'DAVIDSON4-MAY', 'RUTHERFORD2-OCT', 'DAVIDSON4-OCT', 'BENTON6 BIG SANDY-7', 'MCMINNCLAXTON', 'DAVIDSON1-JUL', 'WILLIAMSON1-JUL', 'DAVIDSON4-DEC', 'PERRYPAPER BALLOTS', 'DAVIDSON2-JAN', 'MONROERURAL VALE', 'WILLIAMSON3-MAR', 'WILLIAMSON3-SEP', 'WILLIAMSON1-MAR', 'WAYNE4-2 SH